In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import MinMaxScaler, scale
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import xgboost
from xgboost import XGBRegressor
import sklearn.metrics as metrics
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [3]:
X = pd.read_csv('./dataset-0510/train.csv', index_col='building_id')
X_test = pd.read_csv('./dataset-0510/test.csv', index_col='building_id')

y = X.total_price
X.drop(columns=['total_price'], inplace=True)

In [4]:
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.3, random_state=42) 

In [5]:
# step1. Imputation transformer for completing missing values.
step1 = ('Imputer', Imputer())
# step2. MinMaxScaler
step2 = ('MinMaxScaler', MinMaxScaler())
# step3. feature selection
#step3 = ('FeatureSelection', SelectFromModel(RandomForestRegressor()))
step3 = ('FeatureSelection', VarianceThreshold())

finally_step = ('model',
                XGBRegressor(
                    booster="dart",
                    colsample_bytree=0.4,
                    gamma=0,
                    learning_rate=0.07,
                    max_depth=3,
                    min_child_weight=1.5,
                    n_estimators=500,                                                                    
                    reg_alpha=0.75,
                    reg_lambda=0.45,
                    subsample=0.6,
                    seed=42,
                    objective="reg:squarederror",
                    tree_method='gpu_hist',
                    predictor='gpu_predictor',
                )
               )

pipeline = Pipeline(steps=[step1, step2, finally_step])

pipeline.fit(X_train, y_train)

/home/islab/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


Pipeline(memory=None,
         steps=[('Imputer',
                 Imputer(axis=0, copy=True, missing_values='NaN',
                         strategy='mean', verbose=0)),
                ('MinMaxScaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('model',
                 XGBRegressor(base_score=0.5, booster='dart',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=0.4, gamma=0,
                              importance_type='gain', learning_rate=0.07,
                              max_delta_step=0, max_depth=3,
                              min_child_weight=1.5, missing=None,
                              n_estimators=500, n_jobs=1, nthread=None,
                              objective='reg:squarederror',
                              predictor='gpu_predictor', random_state=0,
                              reg_alpha=0.75, reg_lambda=0.45,
                              scale_pos_weight=1, seed=42, 

## Train score

In [6]:
print(f"Train score: {pipeline.score(X_train, y_train)}")

Train score: 0.9722496110460049


In [7]:
y_pred = pipeline.predict(X_train)
y_pred

array([ 1.4180815e+06,  1.5595130e+07,  4.9857880e+06, ...,
        2.8399824e+08,  2.3922288e+06, -3.6853469e+05], dtype=float32)

In [8]:
#RMSLE = metrics.mean_squared_log_error(y_eval ,y_pred)
VARIANCE_SCORE = metrics.explained_variance_score(y_train, y_pred)
MSLE = np.sqrt(metrics.mean_squared_error(y_train, y_pred))
r2 = metrics.r2_score(y_train, y_pred)

print("MSLE = ", MSLE)
#print("RMSLE = ", RMSLE)
print("VARIANCE_SCORE = ", VARIANCE_SCORE)
print("R2", r2)
'''
Stand score

Train score: 0.9573181748061892
MSLE =  11238714.006184377
VARIANCE_SCORE =  0.9573181796966825
R2 0.9573181748061892
'''

MSLE =  9062117.014496123
VARIANCE_SCORE =  0.972249617429596
R2 0.9722496110460049


'\nStand score\n\nTrain score: 0.9573181748061892\nMSLE =  11238714.006184377\nVARIANCE_SCORE =  0.9573181796966825\nR2 0.9573181748061892\n'

## Test score

In [9]:
print(f"Test score: {pipeline.score(X_eval, y_eval)}")

Test score: 0.7555002700005814


In [10]:
y_pred = pipeline.predict(X_eval)
y_pred

array([ 6297347.5,  3711377. , 12150800. , ...,  6103431. , 43476030. ,
       20158032. ], dtype=float32)

In [11]:
#RMSLE = metrics.mean_squared_log_error(y_eval ,y_pred)
VARIANCE_SCORE = metrics.explained_variance_score(y_eval, y_pred)
MSLE = np.sqrt(metrics.mean_squared_error(y_eval, y_pred))
r2 = metrics.r2_score(y_eval, y_pred)

print("MSLE = ", MSLE)
#print("RMSLE = ", RMSLE)
print("VARIANCE_SCORE = ", VARIANCE_SCORE)
print("R2", r2)

'''
Stand score

Test score: 0.6710864790042561
MSLE =  32747921.6326721
VARIANCE_SCORE =  0.6711062838835371
R2 0.6710864790042561
'''

MSLE =  28234628.05164699
VARIANCE_SCORE =  0.7555296262830761
R2 0.7555002700005814


'\nStand score\n\nTest score: 0.6710864790042561\nMSLE =  32747921.6326721\nVARIANCE_SCORE =  0.6711062838835371\nR2 0.6710864790042561\n'

In [98]:
# save test predictions to file
predictions = pipeline.predict(X_test)
output = pd.DataFrame({'building_id': X_test.index, 'total_price': predictions})
output.to_csv('submission/XGB_predict.csv', index=False)

In [ ]:
output